In [1]:
import re # Regular expressions
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm # Shows a smart progress meter - wrap iterable with tqdm(iterable)!

import collections
from IPython.core.display import HTML
plt.style.reload_library()

## Import Data-Set

Later should be improved: Load all data from a Folder (training Data)

In [260]:
df_raw = pd.read_csv('2020_06_23_CE-BVerwG_DE_Datensatz.csv')
df_raw_use = df_raw.iloc[:5000][["Gericht", "doc_id", "text"]] # Later use full data-set 
#df_raw_use = df_raw.iloc[["Gericht", "doc_id", "text"]]

In [3]:
df_raw

,Unnamed: 0,doc_id,text,Gericht,Datum,Entscheidungsart,Senat,Verfahrensart,Ordinalzahl,Eingangsjahr,Suffix
0,1,BVerwG_1997-02-26_U_6_C_3_96_NA.txt,BUNDESVERWALTUNGSGERICHT\nIM NAMEN DES VOLKES\...,BVerwG,1997-02-26,U,6,C,3,96,NaN
1,2,BVerwG_2000-12-14_B_21_WB_107_00_NA.txt,BUNDESVERWALTUNGSGERICHT\nBeschluss\nBVerwG 1 ...,BVerwG,2000-12-14,B,21,WB,107,0,NaN
2,3,BVerwG_2002-01-10_B_9_A_9_02_NA.txt,BUNDESVERWALTUNGSGERICHT\nBESCHLUSS\nBVerwG 9 ...,BVerwG,2002-01-10,B,9,A,9,2,NaN
3,4,BVerwG_2002-01-14_B_4_BN_1_02_NA.txt,BUNDESVERWALTUNGSGERICHT\nBESCHLUSS\nBVerwG 4 ...,BVerwG,2002-01-14,B,4,BN,1,2,NaN
4,5,BVerwG_2002-01-17_B_1_B_12_02_NA.txt,BUNDESVERWALTUNGSGERICHT\nBESCHLUSS\nBVerwG 1 ...,BVerwG,2002-01-17,B,1,B,12,2,NaN
...,...,...,...,...,...,...,...,...,...,...,...
24219,24220,BVerwG_2020-05-07_B_2_B_35_19_NA.txt,BESCHLUSS\nBVerwG 2 B 35.19\nOVG 4 B 33.14\n\n...,BVerwG,2020-05-07,B,2,B,35,19,NaN
24220,24221,BVerwG_2020-05-12_B_4_BN_3_20_NA.txt,BESCHLUSS\nBVerwG 4 BN 3.20\nOVG 2 C 285/18\n\...,BVerwG,2020-05-12,B,4,BN,3,20,NaN
24221,24222,BVerwG_2020-05-13_B_6_PKH_6_19_NA.txt,BESCHLUSS\nBVerwG 6 PKH 6.19\n\nIn der Verwalt...,BVerwG,2020-05-13,B,6,PKH,6,19,NaN
24222,24223,BVerwG_2020-05-18_B_1_B_21_20_NA.txt,"BESCHLUSS\nBVerwG 1 B 21.20, 1 PKH 12.20\nVGH ...",BVerwG,2020-05-18,B,1,B,21,20,NaN


## Counting occurrences
Count occurrences like “VwGO” (or Tokens of interest) in all decisions either explicitly (exact comparison) or implicitly (ignore case): 

In [4]:
def search_occurrence(word_string, explicit = True): # helper function
    count = 0
    search = str(word_string)
    
    if explicit == True: # search for the exact matching of strings
        for decision in tqdm(df_raw_use['text']):
            count += len(re.findall(search, decision))
    
    else:
        for decision in tqdm(df_raw_use['text']): # ignore the lower / upper case variance in string
            count += len(re.findall(search, decision, re.IGNORECASE))
    
    print('Counts of “%s“: ' % search, count)

In [5]:
search_occurrence("§.*VwGO", False) # False = ignore case

100%|███████████████████████████████████| 1000/1000 [00:00<00:00, 25921.81it/s]

Counts of “§.*VwGO“:  2940


# Regular expressions approach

Output should be a pd.Dataframe with colums:

Document id: Where to find the found reference or string occurrence

Gericht: Which court was the case assigned to

Book: Which book of legislation (if not found return None)

Section: Which section within the book (return 0 if non-existent)

Paragraph: --

Document index: Index of Clausel (Should be a tupel (start : end))

Extracted Clausel: The Clausel of length n which was found to be of interest


In [6]:
df_regex = pd.DataFrame(columns = ["Doc_id", "Court", "Book", "Section", "Paragraph", "Doc_index", "Clausel"])
df_regex


,Doc_id,Court,Book,Section,Paragraph,Doc_index,Clausel


In [7]:
for i in range(5): # len(df_raw_use["text"])
    
    df_raw_use["text"][i]

## N-Gram approach 

Throw N-Gram at the complete Corpus after tokinazation.

Use N-Gram for the sentence after a found start of reference. (Combine Regex with N-Gram to save computing cost, more precice solution?)

Approach: 

Preprocess corpus (Tokenization, ...), then check for N-Grams.

Always retrace to the original sentence. append original sentence instead of reduced form.

### Tokenize

In [261]:
import nltk
import gensim
from gensim.summarization.textcleaner import split_sentences
from gensim.utils import tokenize
import itertools

import codecs

# Create Copy of Data-set
df_preproc = df_raw_use.copy() # preprocessed reduced Dataset

# create Sentences

for instance in tqdm(range(len(df_preproc['text']))):
    
    sentences = nltk.sent_tokenize(df_preproc['text'][instance],language='german') #sentence
    sentences_tok = [nltk.tokenize.word_tokenize(sent) for sent in sentences] # Tokenized
    accumulated = list(itertools.chain.from_iterable(sentences_tok)) # Merge List
    
    accumulated_sent = [accumulated.pop(i) for i in reversed(range(len(accumulated))) if (len(accumulated[i]) != 1)]  # Filter the Tokens for short tokens
    
    df_preproc['text'][instance] = accumulated_sent # assign tokens back to the Data-frame 


100%|██████████████████████████████████████| 5000/5000 [02:25<00:00, 34.38it/s]


In [262]:


# from gensim import corpora 
# import pprint
# # storing the extracted tokens into the dictionary" 
# dictionary = corpora.Dictionary(tqdm(df_preproc['text'])) 
# pprint.pprint(dictionary.token2id)#token -> tokenId.

df_preproc['text'][1]


['Braunmiller',
 'Heimburg',
 'von',
 'Dr.',
 'Lukasczyk',
 'Honnacker',
 'Dr.',
 'Maiwald',
 'Dr.',
 'erachtet',
 'gegeben',
 'für',
 'nicht',
 'WBO',
 'Abs',
 '20',
 'des',
 'Voraussetzungen',
 'die',
 'Senat',
 'der',
 'da',
 'Betracht',
 'in',
 'nicht',
 'kommt',
 'Verfahrenskosten',
 'mit',
 'Antragstellers',
 'des',
 'Belastung',
 'Eine',
 'worden',
 'erteilt',
 'nicht',
 'zufolge',
 'Vorbringen',
 'eigenen',
 'seinem',
 'Antragsteller',
 'dem',
 'ist',
 'werden',
 'zu',
 'versetzt',
 'Dienstposten',
 'begehrten',
 'den',
 'auf',
 'Zusage',
 'rechtsverbindliche',
 'Eine',
 'durfte',
 'werden',
 'gelegt',
 'zugrunde',
 'Verwendungsentscheidung',
 'der',
 'und',
 'ist',
 'geworden',
 'bestandskräftig',
 'Beurteilung',
 'die',
 'dass',
 'Folge',
 'der',
 'mit',
 'getan',
 'nicht',
 'er',
 'hat',
 'Dies',
 'müssen',
 'anfechten',
 'Beschwerde',
 'förmlichen',
 'einer',
 'mit',
 '20/6',
 'ZDv',
 'Buchst',
 '1102',
 'Nr',
 'gemäß',
 'Antragsteller',
 'der',
 'sie',
 'hätte',
 'können',

### Co-occurance probability (of single Words)

In [296]:
# https://medium.com/analytics-vidhya/a-comprehensive-guide-to-build-your-own-language-model-in-python-5141b3917d6d
from nltk import bigrams, trigrams, ngrams
from collections import Counter, defaultdict

# Create a placeholder for model
model = defaultdict(lambda: defaultdict(lambda: 0))

# Count frequency of co-occurance  
for sentence in tqdm(df_preproc['text']):
    for word1, word2, word3 in trigrams(sentence, pad_right=True, pad_left=True):
        model[(word1, word2)][word3] += 1
        
# Probability of a word, given the previous two words
for w1_w2 in tqdm(model):
    total_count = float(sum(model[w1_w2].values()))
    for word3 in model[w1_w2]:
        model[w1_w2][word3] /= total_count
print("Number of dictionary entries:", len(model))

100%|████████████████████████████| 1010847/1010847 [00:03<00:00, 269156.15it/s]

Number of dictionary entries: 1010847


In [264]:
dict(model["Berufungsgericht", "dem"])

{'sich': 0.33771929824561403,
 'vor': 0.18640350877192982,
 'die': 0.02850877192982456,
 'Beschwerde': 0.02850877192982456,
 'wirft': 0.019736842105263157,
 'Sachlage': 0.0021929824561403508,
 'der': 0.006578947368421052,
 'dass': 0.021929824561403508,
 'Kläger': 0.010964912280701754,
 'es': 0.021929824561403508,
 'Beweiserhebung': 0.0021929824561403508,
 'von': 0.10526315789473684,
 'hält': 0.046052631578947366,
 'gegenüber': 0.008771929824561403,
 'und': 0.013157894736842105,
 'sie': 0.0043859649122807015,
 'Anhörung': 0.0043859649122807015,
 'hier': 0.0043859649122807015,
 'Das': 0.006578947368421052,
 'vielmehr': 0.0021929824561403508,
 'hat': 0.0043859649122807015,
 'Beigeladene': 0.006578947368421052,
 'mit': 0.02631578947368421,
 'offenbar': 0.0021929824561403508,
 'was': 0.0021929824561403508,
 'das': 0.0021929824561403508,
 'Aufklärung': 0.0043859649122807015,
 'dort': 0.0021929824561403508,
 'Hinwirken': 0.0021929824561403508,
 '2002': 0.0021929824561403508,
 'Aufklärungen': 

### repeat for n-Grams following a N-Gram 

In [326]:
# Create a placeholder for model
model_2 = defaultdict(lambda: defaultdict(lambda: 0))

# Create trigrams and repeat process

df_preproc_trig = df_preproc.copy()

def get_ngrams(text_input, order, Keep_duplicates = True):
    for instance in tqdm(range(len(text_input))):
        sentence = text_input[instance]
        n_grams = ngrams(sentence, order)
        output = [ ' '.join(grams) for grams in n_grams]
        
        if Keep_duplicates == True:
            text_input[instance] = output
        else:
            text_input[instance] = output[::order]
    
    
get_ngrams(df_preproc_trig['text'], order=2, Keep_duplicates=False)

# Count frequency of co-occurance  
for sentence in tqdm(df_preproc_trig['text']):
    for word1, word2, word3 in trigrams(sentence, pad_right=True, pad_left=True):
        model_2[(word1, word2)][word3] += 1
        
# Probability of a word, given the previous two words
for w1_w2 in tqdm(model_2):
    total_count = float(sum(model_2[w1_w2].values()))
    for word3 in model_2[w1_w2]:
        model_2[w1_w2][word3] /= total_count
print("Number of dictionary entries:", len(model_2))

100%|████████████████████████████| 1594954/1594954 [00:04<00:00, 350568.63it/s]

Number of dictionary entries: 1594954


In [328]:
dict(model_2["vom Senats", "des Urteil"])

{'378 Nr': 0.003289473684210526,
 'hat gewählt': 0.003289473684210526,
 'dem von': 0.049342105263157895,
 'unterziehen zu': 0.003289473684210526,
 'Tatsacheninstanz der': 0.003289473684210526,
 'dem bereits': 0.003289473684210526,
 'vgl waren': 0.003289473684210526,
 'ist zuzulassen': 0.003289473684210526,
 'dem in': 0.02631578947368421,
 'abweist insoweit': 0.003289473684210526,
 'dem den': 0.003289473684210526,
 'Nr BSHG': 0.003289473684210526,
 'zuzuordnen Versteigerungsgutes': 0.003289473684210526,
 'hat überzeugen': 0.003289473684210526,
 'ist wirksam': 0.003289473684210526,
 'werden versagt': 0.003289473684210526,
 'Psychose hebephrener': 0.003289473684210526,
 'im den': 0.003289473684210526,
 'bleiben erschlossen': 0.003289473684210526,
 'VwVfGBbg Satz': 0.003289473684210526,
 'ist Verfügungsberechtigtem': 0.003289473684210526,
 'ausgeschlossen ist': 0.003289473684210526,
 'bilden zu': 0.006578947368421052,
 '58 39': 0.003289473684210526,
 'zulässig Revision': 0.0032894736842105

## Exporting

Accumulate entires within the df, add accumulation column.





Export DataFrames (raw and accumulated) to .csv



## Vizualization

Plot most occuring sentences: compare N-Gram to Regex approach